In [7]:
import pandas as pd
import numpy as np
import dgl
import createGraph
from importlib import reload
import torch
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import architecture
import training_loop

In [8]:
def process_data(filename):
    # Load Data
    df = pd.read_csv(filename)
    X = df.drop(labels=["PostCond"], axis=1).to_numpy()
    y = df["PostCond"].to_numpy()

    # impute missing values
    imputer = IterativeImputer(random_state=42)
    X = imputer.fit_transform(X)

    # convert imputed ndarrays back into a Dataframe
    imputed_X_df = pd.DataFrame(data=X, columns=df.drop(labels=["PostCond"], axis=1).columns)
    y_df = pd.DataFrame(data=y, columns=["PostCond"])
    imputed_df = pd.concat([imputed_X_df, y_df], axis=1)

    # Generate graphs
    g, node_batches, y = createGraph.throughData(imputed_df)

    return g, node_batches, y

In [9]:
# Assign new indices to 2nd graph
def offset_node_batches(node_batches, offset):
    return [[idx + offset for idx in batch] for batch in node_batches]

In [10]:
train_g, train_node_batches, train_y = process_data('upsampledTrain.csv')
val_g, val_node_batches, val_y = process_data('nonupsampledTest.csv')

Patient 0.0
Patient 2.0
Patient 17.0
Patient 25.0
Number of edges: 23
Patient 26.0
Patient 28.0
Number of edges: 4
Patient 39.0
Number of edges: 3
Patient 41.0
Number of edges: 1
Patient 42.0
Patient 44.0
Patient 46.0
Patient 47.0
Patient 49.0
Patient 53.0
Number of edges: 2
Patient 54.0
Number of edges: 4
Patient 59.0
Number of edges: 1
Patient 65.0
Number of edges: 1
Patient 66.0
Patient 67.0
Number of edges: 2
Patient 68.0
Patient 70.0
Patient 71.0
Number of edges: 7
Patient 72.0
Number of edges: 1
Patient 78.0
Number of edges: 1
Patient 81.0
Patient 85.0
Number of edges: 1
Patient 86.0
Number of edges: 1
Patient 91.0
Number of edges: 1
Patient 92.0
Patient 95.0
Number of edges: 4
Patient 96.0
Number of edges: 13
Patient 100.0
Number of edges: 7
Patient 103.0
Patient 106.0
Patient 116.0
Number of edges: 3
Patient 117.0
Number of edges: 3
Patient 119.0
Number of edges: 1
Patient 120.0
Number of edges: 4
Patient 122.0
Number of edges: 2
Patient 128.0
Patient 129.0
Number of edges: 1
P

In [11]:
# Retrieve number of nodes in train graph for offset 
num_train_nodes = train_g.number_of_nodes()
# Add number of train nodes to each idx in val set 
offset_val_node_batches = offset_node_batches(val_node_batches, num_train_nodes)
# Combine into single graph
combined_g = dgl.batch([train_g, val_g])
# Get total number of batches/indices 
combined_node_batches = train_node_batches + offset_val_node_batches
# Combine labels for train and val into single tensor
combined_y = torch.cat([train_y, val_y])
# Mask to select training set
train_mask = list(range(len(train_y)))
# Mask to select val set
val_mask = list(range(len(train_y), len(train_y) + len(val_y)))

In [12]:
import architecture
import training_loop
reload(architecture)
reload(training_loop)

model = architecture.GCN(train_g.ndata['feat'].shape[1], 30, 30)

training_loop.train(g=combined_g, node_batches=combined_node_batches, model=model, labels=combined_y, train_mask=train_mask, val_mask=val_mask, validate=True, test=False)

C:\Users\james\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'novograd' from 'timm.optim' (C:\Users\james\AppData\Roaming\Python\Python311\site-packages\timm\optim\__init__.py)